# Introduction to chemiscope

This notebook contains an introduction to the capabilities of
[chemiscope](https://chemiscope.org/), the interactive structure-properties
explorer for materials and molecules. 

You can run this notebook after cloning the corresponding code and installing
the dependencies:
```
git clone https://github.com/Luthaf/Ringberg-SummerSchool2022
cd Ringberg-SummerSchool2022
pip install -r requirements.txt
jupyter notebook
```

In [ ]:
import ase.io
import numpy as np
import matplotlib.pyplot as plt

import chemiscope

Chemiscope is a tool to explore atomistic datasets: simulation trajectories,
database of calculations for high-throughput screening, training set for machine
learning, etc. It allows the interactive exploration of these large dataset,
helping to guide the human intuition when looking for correlations in the data,
as well as structure-property relationships.

For this tutorial, we will be using the QM7 dataset. It is a dataset of small
organic molecules (up to 7 non-hydrogen atoms), which is provided in the
extended XYZ format in the same repository as this notebook. This dataset
contains a single physical property: the atomization energy associated with each
structure.

In [ ]:
frames = ase.io.read("qm7.xyz", ":200")
print(f"we are using {len(frames)} structures from the QM7 dataset")

atomization_energies = np.array([frame.info["atomization_energy"] for frame in frames]).reshape(-1, 1)

## Structural representations

If we want to correlate the atomic structure and the properties, we need to be able to talk about structures and in particular structure similarity in an objective way.

One possible tool to do this is one of the dozen atomistic representation[1] that was created in the last years to be used with machine learning. Here we will use the SOAP power spectrum[2], but other choices are possible and interesting depending what you want to do.

[1] - Physics-Inspired Structural Representations for Molecules and Materials, [10.1021/acs.chemrev.1c00021](https://doi.org/10.1021/acs.chemrev.1c00021)

[2] - On representing chemical environments, [10.1103/PhysRevB.87.184115](https://doi.org/10.1103/PhysRevB.87.184115)

In [ ]:
from rascaline import SoapPowerSpectrum

calculator = SoapPowerSpectrum(
    cutoff=3.5,
    max_radial=6,
    max_angular=6,
    atomic_gaussian_width=0.3,
    center_atom_weight=1.0,
    radial_basis={"Gto": {}},
    cutoff_function={"ShiftedCosine": {"width": 0.5}},
    gradients=False,
)

def average_over_structures(descriptor):
    samples = descriptor.block().samples
    values = descriptor.block().values
    
    all_structures = np.unique(samples["structure"])
    
    output = np.zeros((len(all_structures), values.shape[1]))
    for i, structure in enumerate(all_structures):
        mask = samples["structure"] == structure
        output[i, :] += np.mean(values[mask, :], axis=0)
    
    return output

In [ ]:
# compute the SOAP power spectrum using rascaline, and transform it from the
# species-sparse storage to the usual single matrix storage

per_atom_soap = calculator.compute(frames)
per_atom_soap.keys_to_samples("species_center")
per_atom_soap.keys_to_properties(["species_neighbor_1", "species_neighbor_2"])

per_structure_soap = average_over_structures(per_atom_soap)
per_atom_soap = per_atom_soap.block().values

Unfortunately, the resulting SOAP representation is very high-dimensional (3528 dimensions in this specific case), making it impossible to interpret and understand just by looking at the values.

In [ ]:
print(per_structure_soap.shape)
print("CH4", per_structure_soap[0])

We will want to use dimensionality reduction algorithms to go from 3528 dimensions to something the human brain can apprehend, 2 to 4 dimensions.

One of the simplest tool for such dimensionality reduction is the Principal Components Analysis or PCA. This algorithm tries to find the directions of highest variance in the high dimensionality space, and use these as axes for the low dimensionality space. 

We will use sklearn and skcosmo (an extension to sklearn developed in the COSMO laboratory) to prepare the data and compute the PCA

In [ ]:
import sklearn
import sklearn.decomposition

import skcosmo
import skcosmo.decomposition
import skcosmo.preprocessing

In [ ]:
pca = sklearn.decomposition.PCA(n_components=4)
scaler = skcosmo.preprocessing.StandardFlexibleScaler(copy=True)

X = scaler.fit_transform(per_structure_soap)
soap_pca = pca.fit_transform(X)

pca.explained_variance_ratio_

Now we can try to look for correlations between the structural features and the property

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 8))
ax[0].scatter(soap_pca[:, 0], atomization_energies)
ax[0].set_xlabel("PCA axis 1")
ax[0].set_ylabel("atomization energy / kcal/mol")


ax[1].scatter(soap_pca[:, 1], atomization_energies)
ax[1].set_ylabel("PCA axis 2")
ax[1].set_ylabel("atomization energy / kcal/mol")
plt.show()

These plots are relatively hard to understand (which point correspond to which structure?), and are limited to 2 dimensions. That's where chemiscope enters the stage!

In [ ]:
chemiscope.show(
    frames=frames,
    properties={
        "SOAP PCA": soap_pca,
    },
    settings={"map": {"color": {"property": "atomization_energy"}}}
)

Here we can see how similar structures ends up close to one another in the projected PCA space, and there is some correlation between the SOAP representation and the atomization energy of the molecules

In [ ]:
chemiscope.show(
    frames=frames,
    properties={
        "SOAP PCA": soap_pca,
    },
    settings={"map": {"color": {"property": "atomization_energy"}}, "pinned": [26, 150, 128, 182]}
)

## PCovR: increasing correlations between the projection and the properties

The above projection of SOAP somehow correlates with the energies, but not very well. If we want to find out more about structure-properties relatioships, we can biais the projection to be closer to a linear regression. This is the idea behing the Principal Covariate Regression method (PCovR), which is implemented in skcosmo.

See "Structure-property maps with Kernel principal covariates regression" ([10.1088/2632-2153/aba9ef](https://doi.org/10.1088/2632-2153/aba9ef)) for more information on PCA, PCovR and kernel extension of these methods.

In [ ]:
pcovr_0 = skcosmo.decomposition.PCovR(n_components=4, mixing=0.0)
pcovr_5 = skcosmo.decomposition.PCovR(n_components=4, mixing=0.5)
pcovr_9 = skcosmo.decomposition.PCovR(n_components=4, mixing=0.9)

X = scaler.fit_transform(per_structure_soap)
y = scaler.fit_transform(atomization_energies)
soap_pcovr0 = pcovr_0.fit_transform(X, y)
soap_pcovr5 = pcovr_5.fit_transform(X, y)
soap_pcovr9 = pcovr_9.fit_transform(X, y)

In [ ]:
properties = {
    "SOAP PCovR α=0.0": soap_pcovr0,
    "SOAP PCovR α=0.5": soap_pcovr5,
    "SOAP PCovR α=0.9": soap_pcovr9,
}

chemiscope.show(
    frames=frames,
    properties=properties,
    settings={'map': {'color': {'property': 'atomization_energy'}}},
)

## Per-atom properties

Chemiscope is also able to display per-atom properties and structure views.

We will start with a small detour, training a linear model on summed (not averaged) SOAP
vectors. We can then use this model with the per-atom SOAP representation to
predict per-atom energies to use in chemiscope.

In [ ]:
def sum_over_structures(descriptor):
    samples = descriptor.block().samples
    values = descriptor.block().values
    
    all_structures = np.unique(samples["structure"])
    
    output = np.zeros((len(all_structures), values.shape[1]))
    for i, structure in enumerate(all_structures):
        mask = samples["structure"] == structure
        output[i, :] += np.sum(values[mask, :], axis=0)
    
    return output

per_atom_soap = calculator.compute(frames)
per_atom_soap.keys_to_samples("species_center")
per_atom_soap.keys_to_properties(["species_neighbor_1", "species_neighbor_2"])

per_structure_soap = sum_over_structures(per_atom_soap)
per_atom_soap = per_atom_soap.block().values

In [ ]:
model = sklearn.linear_model.Ridge(alpha=1e-4, fit_intercept=False)
model.fit(per_structure_soap, atomization_energies)

predicted = model.predict(per_structure_soap)

plt.scatter(atomization_energies, predicted)
min_e, max_e = (np.min(atomization_energies), np.max(atomization_energies))
plt.plot([min_e, max_e], [min_e, max_e], color="r")

In [ ]:
local_energies = model.predict(per_atom_soap)

print("per-atom energy in CH4")
print(local_energies[0:5, :])

In [ ]:
X = scaler.fit_transform(per_atom_soap)
y = scaler.fit_transform(local_energies)
local_soap_pcovr = pcovr_5.fit_transform(X, y)

properties = {
    "SOAP PCovR": local_soap_pcovr,
    "local_atom_energy": local_energies,
}

# Create a list of all atom-centered environnements
environments = chemiscope.all_atomic_environments(frames, cutoff=3.5)

chemiscope.show(
    frames=frames,
    properties=properties,
    environments=environments,
    settings={'map': {'color': {'property': 'local_atom_energy'}}},
)

## Sharing the results with others

In [ ]:
# save to file and load into website

chemiscope.write_input(
    "qm7.json.gz",
    frames=frames,
    properties=properties,
    environments=environments,
)

## Other cool things you can do with chemiscope/jupyter

In [ ]:
# view structures only

chemiscope.show(frames=frames, mode="structure")

In [ ]:
# view map only

chemiscope.show(properties=properties, mode="map")

In [ ]:
# set settings

settings = {
    "map": {
        "color": {"property": "local_atom_energy"},
        "z": {"property": "SOAP PCovR[3]"}
    },
    "structure": [
        {"spaceFilling": True},
        {"spaceFilling": False},
    ],
    "pinned": [0, 100]
}
chemiscope.show(frames=frames, properties=properties, environments=environments, settings=settings)